# Tensorboard Basics

Visualización de gráficos y pérdidas mediante Tensorboard. Este ejemplo utiliza la base de datos de dígitos manuscritos del MNIST (http://yann.lecun.com/exdb/mnist/).

In [1]:
import tensorflow as tf

# Importar datos MINST

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parametros
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = '../../logs/'

# tf Entrada de gráficos
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# Reconocimiento de 0-9 dígitos => 10 clases
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Establecer pesos del modelo
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [3]:
# Construir el modelo y encapsular todas las operaciones en alcances, haciendo
# La visualización gráfica de la tensorboard es más conveniente
with tf.name_scope('Model'):
    # Modelo
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimizar los errores mediante la entropía cruzada
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Descenso en pendiente
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Precisión
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Inicializando las variables
init = tf.global_variables_initializer()

# Crear un resumen para monitorear el tensor de costos
tf.summary.scalar("loss", cost)
# Crear un resumen para monitorear el tensor de precisión
tf.summary.scalar("accuracy", acc)
# Fusionar todos los resúmenes en una sola operación
merged_summary_op = tf.summary.merge_all()

In [4]:
# Iniciar Entrenamiento
with tf.Session() as sess:
    sess.run(init)

    # op para escribir registros en la Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Circulo de entrenamiento
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Circuito sobre todos los lotes
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Ejecutar optimización op (backprop), cost op (para obtener valor de pérdida)
            # y nodos de resumen
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Escribe registros en cada iteración
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Calcula la pérdida promedio
            avg_cost += c / total_batch
        # Mostrar registros por paso de época
        if (epoch+1) % display_epoch == 0:
            print(("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)))

    print("Optimization Finished!")

    # Modelo de prueba
    # Calcular la precisión
    print(("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels})))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

('Epoch:', '0001', 'cost=', '1.183388450')
('Epoch:', '0002', 'cost=', '0.665133665')
('Epoch:', '0003', 'cost=', '0.552735593')
('Epoch:', '0004', 'cost=', '0.498670717')
('Epoch:', '0005', 'cost=', '0.465501758')
('Epoch:', '0006', 'cost=', '0.442556846')
('Epoch:', '0007', 'cost=', '0.425509037')
('Epoch:', '0008', 'cost=', '0.412173652')
('Epoch:', '0009', 'cost=', '0.401385705')
('Epoch:', '0010', 'cost=', '0.392384847')
('Epoch:', '0011', 'cost=', '0.384692703')
('Epoch:', '0012', 'cost=', '0.378199260')
('Epoch:', '0013', 'cost=', '0.372414338')
('Epoch:', '0014', 'cost=', '0.367254242')
('Epoch:', '0015', 'cost=', '0.362731107')
('Epoch:', '0016', 'cost=', '0.358590668')
('Epoch:', '0017', 'cost=', '0.354855068')
('Epoch:', '0018', 'cost=', '0.351434447')
('Epoch:', '0019', 'cost=', '0.348324607')
('Epoch:', '0020', 'cost=', '0.345396065')
('Epoch:', '0021', 'cost=', '0.342742144')
('Epoch:', '0022', 'cost=', '0.340254615')
('Epoch:', '0023', 'cost=', '0.337927042')
('Epoch:', 

### Visualización de Pérdidas y Precisión

<img src="../../resources/img/tensorboard_basic_1.png"/>

### Visualización de gráficos

<img src="../../resources/img/tensorboard_basic_2.png"/>